### install necessary dependencies

In [1]:
%pip install sae-lens transformer-lens sae-dashboard huggingface_hub[cli] tabulate openai ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 68.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 33.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 kB 132.1 MB/s eta 0:00:00


In [ ]:
# !pip install accelerate
# in terminal
# apt install unzip

In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import torch
from tabulate import tabulate

import sae_lens
from transformer_lens import HookedTransformer
from sae_lens import SAE,HookedSAETransformer,ActivationsStore
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from torch import Tensor, nn
import einops
from rich import print as rprint
from rich.table import Table
from tqdm.auto import tqdm
import pandas as pd
import requests
from typing import Any, Callable, Literal, TypeAlias
from openai import OpenAI
from huggingface_hub import interpreter_login
import os
import sys


In [9]:
chapter = "chapter1_transformer_interp"
repo = "ARENA_3.0"
root = "/workspace/vads-prevalent-safety-llm/notebooks"

if not os.path.exists(f"{root}/{chapter}"):
    !wget https://github.com/callummcdougall/ARENA_3.0/archive/refs/heads/main.zip
    !unzip {root}/main.zip 'ARENA_3.0-main/chapter1_transformer_interp/exercises/*'
    !mv {root}/{repo}-main/{chapter} {root}/{chapter}
    !rm {root}/main.zip
    !rmdir {root}/{repo}-main

# !touch {root}/chapter1_transformer_interp/exercises/part32_superposition_and_saes/__init__.py
# !touch {root}/chapter1_transformer_interp/exercises/__init__.py

# !touch //chapter1_transformer_interp/exercises/part32_superposition_and_saes/__init__.py
# !touch /content/chapter1_transformer_interp/exercises/__init__.py
sys.path.append(f"{root}/{chapter}/exercises")

In [8]:
import part31_superposition_and_saes.tests as part31_tests
import part31_superposition_and_saes.utils as part31_utils

In [3]:
# notebook_login()
from dotenv import load_dotenv
import os

load_dotenv()

HF_TOKEN=os.getenv("HF_TOKEN")
OPENAI_TOKEN = os.getenv("OPEN_API_KEY")

In [4]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


In [5]:
def display_dashboard(
    sae_release="gpt2-small-res-jb",
    sae_id="blocks.7.hook_resid_pre",
    latent_idx=0,
    width=800,
    height=600,
):
    release = get_pretrained_saes_directory()[sae_release]
    neuronpedia_id = release.neuronpedia_id[sae_id]

    url = f"https://neuronpedia.org/{neuronpedia_id}/{latent_idx}?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"

    print(url)
    display(IFrame(url, width=width, height=height))

In [6]:
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory

# TODO: Make this nicer.
df = pd.DataFrame.from_records(
    {k: v.__dict__ for k, v in get_pretrained_saes_directory().items()}
).T
df.drop(
    columns=[
        "expected_var_explained",
        "expected_l0",
        "config_overrides",
        "conversion_func",
    ],
    inplace=True,
)
# df  # Each row is a "release" which has multiple SAEs which may have different configs / match different hook points in a model.
df.head()

,release,repo_id,model,saes_map,neuronpedia_id
gemma-2b-it-res-jb,gemma-2b-it-res-jb,jbloom/Gemma-2b-IT-Residual-Stream-SAEs,gemma-2b-it,{'blocks.12.hook_resid_post': 'gemma_2b_it_blo...,{'blocks.12.hook_resid_post': 'gemma-2b-it/12-...
gemma-2b-res-jb,gemma-2b-res-jb,jbloom/Gemma-2b-Residual-Stream-SAEs,gemma-2b,{'blocks.0.hook_resid_post': 'gemma_2b_blocks....,{'blocks.0.hook_resid_post': 'gemma-2b/0-res-j...
gemma-scope-27b-pt-res,gemma-scope-27b-pt-res,google/gemma-scope-27b-pt-res,gemma-2-27b,{'layer_10/width_131k/average_l0_106': 'layer_...,"{'layer_10/width_131k/average_l0_106': None, '..."
gemma-scope-27b-pt-res-canonical,gemma-scope-27b-pt-res-canonical,google/gemma-scope-27b-pt-res,gemma-2-27b,{'layer_10/width_131k/canonical': 'layer_10/wi...,{'layer_10/width_131k/canonical': 'gemma-2-27b...
gemma-scope-2b-pt-att,gemma-scope-2b-pt-att,google/gemma-scope-2b-pt-att,gemma-2-2b,{'layer_0/width_16k/average_l0_104': 'layer_0/...,"{'layer_0/width_16k/average_l0_104': None, 'la..."


In [7]:
df.loc[df.release == "gemma-2b-res-jb"]

,release,repo_id,model,saes_map,neuronpedia_id
gemma-2b-res-jb,gemma-2b-res-jb,jbloom/Gemma-2b-Residual-Stream-SAEs,gemma-2b,{'blocks.0.hook_resid_post': 'gemma_2b_blocks....,{'blocks.0.hook_resid_post': 'gemma-2b/0-res-j...


## Load the Model 

In [8]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# Load the LLM
model = HookedSAETransformer.from_pretrained_no_processing(
    "gemma-2-2b",
    device = device,
    torch_dtype = torch.float16,
    device_map = "auto"
)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


## Load SAE

In [9]:
# Load the corresponding SAE
release="gemma-scope-2b-pt-res-canonical"  # Replace with the correct release for your model
sae_id="layer_20/width_16k/canonical"
sae, cfg_dict, _ = sae_lens.SAE.from_pretrained(
    release=release,  # Replace with the correct release for your model
    sae_id=sae_id,
    device=device,
    # device_map = "auto",
)


# # Load the corresponding SAE
# release="gemma-scope-2b-pt-res-canonical"  # Replace with the correct release for your model
# sae_id="layer_12/width_1m/canonical"
# sae, cfg_dict, _ = sae_lens.SAE.from_pretrained(
#     release=release,  # Replace with the correct release for your model
#     sae_id=sae_id,
#     device=device,
#     # device_map = "auto",
# )

params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [10]:
# latent_idx = 12082
latent_idx = 9

display_dashboard(sae_release=release, sae_id=sae_id, latent_idx=latent_idx)

https://neuronpedia.org/gemma-2-2b/20-gemmascope-res-16k/9?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300


## Activation Store Creation

In [10]:
# activation store
gemma2_act_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    streaming=True,
    store_batch_size_prompts=8,
    train_batch_size_tokens=2048,
    n_batches_in_buffer=16,
    device=str(device),
)

# Example of how you can use this:
with torch.no_grad():
    tokens = gemma2_act_store.get_batch_tokens()
assert tokens.shape == (gemma2_act_store.store_batch_size_prompts, gemma2_act_store.context_size)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sae_lens/training/activations_store.py:246: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(


In [12]:
tokens.shape # as the batch is of 8 prompts and each with 1024 context_size

torch.Size([8, 1024])

## Utility Functions 

In [11]:
def get_k_largest_indices(
    x: Float[Tensor, "batch seq"],
    k: int,
    buffer: int = 0,
    no_overlap: bool = True,
) -> Int[Tensor, "k 2"]:
    """
    Returns the tensor of (batch, seqpos) indices for each of the top k elements in the tensor x.

    Args:
        buffer:     We won't choose any elements within `buffer` from the start or end of their seq (this helps if we
                    want more context around the chosen tokens).
        no_overlap: If True, this ensures that no 2 top-activating tokens are in the same seq and within `buffer` of
                    each other.
    """
    assert buffer * 2 < x.size(1), "Buffer is too large for the sequence length"
    assert not no_overlap or k <= x.size(0), "Not enough sequences to have a different token in each sequence"

    if buffer > 0:
        x = x[:, buffer:-buffer]

    indices = x.flatten().argsort(-1, descending=True)
    # extra_buffer = 10
    # values, indices = x.flatten().topk(k + extra_buffer,largest = True)
    rows = indices // x.size(1)
    cols = indices % x.size(1) + buffer

    if rows.numel() == 0 or cols.numel() ==0:
        raise ValueError("No Valid activations found after applying buffer.")

    if no_overlap:
        unique_indices = torch.empty((0, 2), device=x.device).long()
        while len(unique_indices) < k:
            unique_indices = torch.cat((unique_indices, torch.tensor([[rows[0], cols[0]]], device=x.device)))
            is_overlapping_mask = (rows == rows[0]) & ((cols - cols[0]).abs() <= buffer)
            rows = rows[~is_overlapping_mask]
            cols = cols[~is_overlapping_mask]
        return unique_indices

    return torch.stack((rows, cols), dim=1)[:k]

# x = torch.arange(40, device=device).reshape((2, 20))
# x[0, 10] += 50  # 2nd highest value
# x[0, 11] += 100  # highest value
# x[1, 1] += 150  # not inside buffer (it's less than 3 from the start of the sequence)
# top_indices = get_k_largest_indices(x, k=2, buffer=3)
# rprint(top_indices)
# assert top_indices.tolist() == [[0, 11], [0, 10]]


In [12]:


def index_with_buffer(
    x: Float[Tensor, "batch seq"], indices: Int[Tensor, "k 2"], buffer: int | None = None
) -> Float[Tensor, "k *buffer_x2_plus1"]:
    """
    Indexes into `x` with `indices` (which should have come from the `get_k_largest_indices` function), and takes a
    +-buffer range around each indexed element. If `indices` are less than `buffer` away from the start of a sequence
    then we just take the first `2*buffer+1` elems (same for at the end of a sequence).

    If `buffer` is None, then we don't add any buffer and just return the elements at the given indices.
    """
    rows, cols = indices.unbind(dim=-1)
    if buffer is not None:
        rows = einops.repeat(rows, "k -> k buffer", buffer=buffer * 2 + 1)
        cols[cols < buffer] = buffer
        cols[cols > x.size(1) - buffer - 1] = x.size(1) - buffer - 1
        cols = einops.repeat(cols, "k -> k buffer", buffer=buffer * 2 + 1) + torch.arange(
            -buffer, buffer + 1, device=cols.device
        )
    return x[rows, cols]


# x_top_values_with_context = index_with_buffer(x, top_indices, buffer=3)
# assert x_top_values_with_context[0].tolist() == [8, 9, 10 + 50, 11 + 100, 12, 13, 14]  # highest value in the middle
# assert x_top_values_with_context[1].tolist() == [7, 8, 9, 10 + 50, 11 + 100, 12, 13]  # 2nd highest value in the middle

In [14]:


def display_top_seqs(data: list[tuple[float, list[str], int]]):
    """
    Given a list of (activation: float, str_toks: list[str], seq_pos: int), displays a table of these sequences, with
    the relevant token highlighted.

    We also turn newlines into "\\n", and remove unknown tokens � (usually weird quotation marks) for readability.
    """
    table = Table("Act", "Sequence", title="Max Activating Examples", show_lines=True)
    for act, str_toks, seq_pos in data:
        formatted_seq = (
            "".join([f"[b u green]{str_tok}[/]" if i == seq_pos else str_tok for i, str_tok in enumerate(str_toks)])
            .replace("�", "")
            .replace("\n", "↵")
        )
        table.add_row(f"{act:.3f}", repr(formatted_seq))
    rprint(table)


example_data = [
    (0.5, [" one", " two", " three"], 0),
    (1.5, [" one", " two", " three"], 1),
    (2.5, [" one", " two", " three"], 2),
]
display_top_seqs(example_data)

  Max Activating Examples   
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Act   ┃ Sequence         ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 0.500 │ ' one two three' │
├───────┼──────────────────┤
│ 1.500 │ ' one two three' │
├───────┼──────────────────┤
│ 2.500 │ ' one two three' │
└───────┴──────────────────┘

In [15]:
def fetch_max_activating_examples(
    model: HookedSAETransformer,
    sae: SAE,
    act_store: ActivationsStore,
    latent_idx: int,
    total_batches: int = 100,
    k: int = 10,
    buffer: int = 10,
    display: bool = False,
) -> list[tuple[float, list[str], int]]:
    """
    Displays the max activating examples across a number of batches from the
    activations store, using the `display_top_seqs` function.
    """
    sae_acts_post_hook_name = f"{sae.cfg.hook_name}.hook_sae_acts_post"

    # Create list to store the top k activations for each batch. Once we're done,
    # we'll filter this to only contain the top k over all batches
    data = []

    with torch.no_grad():
        for _ in range(total_batches):
            tokens = act_store.get_batch_tokens(batch_size = 5)
            # Handling empty batch
            if tokens is None or tokens.numel() == 0:
                continue

            tokens = tokens.to(model.cfg.device)
            # print("Tokens shape:", tokens.shape)

            batch_size = tokens.size(0)
            current_k = min(k,batch_size)
    
            _, cache = model.run_with_cache_with_saes(
                tokens,
                saes=[sae],
                stop_at_layer=sae.cfg.hook_layer + 1,
                names_filter=[sae_acts_post_hook_name],
            )
            acts = cache[sae_acts_post_hook_name][..., latent_idx]
            # print("Activations shape:", acts.shape)
            # Get largest indices, get the corresponding max acts, and get the surrounding indices
            k_largest_indices = get_k_largest_indices(acts, k=current_k, buffer=buffer,no_overlap = True)
            # print(k_largest_indices)
            tokens_with_buffer = index_with_buffer(tokens, k_largest_indices, buffer=buffer)
            str_toks = [model.to_str_tokens(toks) for toks in tokens_with_buffer]
            top_acts = index_with_buffer(acts, k_largest_indices).tolist()
            data.extend(list(zip(top_acts, str_toks, [buffer] * len(str_toks))))
    
            # GPU cache clear
        torch.cuda.empty_cache()


    data = sorted(data, key=lambda x: x[0], reverse=True)[:k]
    if display:
        display_top_seqs(data)
    return data


# Display your results, and also test them
# buffer = 5
# data = fetch_max_activating_examples(model, sae, gemma2_act_store, latent_idx=9, buffer=buffer, k=5, display=True)
# first_seq_str_tokens = data[0][1]
# assert first_seq_str_tokens[buffer] == " Fight"

## Autointerp

In [73]:
def get_autointerp_df(sae_release="gpt2-small-res-jb", sae_id="blocks.7.hook_resid_pre") -> pd.DataFrame:
    release = get_pretrained_saes_directory()[sae_release]
    neuronpedia_id = release.neuronpedia_id[sae_id]

    url = "https://www.neuronpedia.org/api/explanation/export?modelId={}&saeId={}".format(*neuronpedia_id.split("/"))
    headers = {"Content-Type": "application/json"}
    response = requests.get(url, headers=headers)

    data = response.json()
    return pd.DataFrame(data)


explanations_df_gemma_2b = get_autointerp_df(sae_release = release,sae_id = sae_id)
explanations_df_gemma_2b.head()

,modelId,layer,index,description,explanationModelName,typeName
0,gemma-2-2b,20-gemmascope-res-16k,14403,"phrases or sentences that introduce lists, exa...",claude-3-5-sonnet-20240620,oai_token-act-pair
1,gemma-2-2b,20-gemmascope-res-16k,14403,references to numerical sports scores and resu...,gemini-1.5-pro,oai_token-act-pair
2,gemma-2-2b,20-gemmascope-res-16k,14403,text related to sports accomplishments and sta...,gpt-4o-mini,oai_token-act-pair
3,gemma-2-2b,20-gemmascope-res-16k,10131,phrases referring to being fluent in a languag...,gemini-1.5-flash,oai_token-act-pair
4,gemma-2-2b,20-gemmascope-res-16k,10133,words related to scientific studies and proces...,gemini-1.5-flash,oai_token-act-pair


In [74]:
# example 
# df_temp = get_autointerp_df(release,sae_id)
# df_temp[df_temp['explanationModelName'] == 'gpt-4o-mini']
# df_temp.loc[df_temp['index'] == '6631', 'description'].iloc[0]

In [75]:
explanations_df_gemma_2b = get_autointerp_df(sae_release = release,sae_id = sae_id)

def get_autointerp_explanation_df(
    explanations_df: pd.DataFrame,
    latent_idx: int
) -> str:
    if explanations_df.empty:
        raise ValueError("The explanations DataFrame is empty.")

    if latent_idx not in explanations_df['index'].values:
        raise ValueError(f"Latent index {latent_idx} not found in the explanations DataFrame.")

    return explanations_df.loc[
        explanations_df['index'] == latent_idx, ['description','explanationModelName']
    ].iloc[0]


In [48]:
completions = get_autointerp_explanation_df(explanations_df_gemma_2b,latent_idx='4442')
print(completions.description)

instances of the word "kill" and its variations, highlighting themes of violence and death


## Top Activating Latents

In [70]:
def get_top_activating_latents(
    model: HookedSAETransformer,
    sae: SAE,
    act_store: ActivationsStore,
    prompt: str,
    k: int = 10,
    top_n: int = 15
) -> list[tuple[int, float]]:
    """
    Runs a given prompt through the model and SAE, and returns the top `k` activating latents.

    Args:
        model: The HookedSAETransformer model with SAE hooks.
        sae: The Sparse Autoencoder (SAE) for encoding activations.
        act_store: The ActivationsStore for managing cached activations.
        prompt: The input prompt to analyze.
        k: Number of top activating latents to return (default is 10).

    Returns:
        A list of tuples (latent_id, activation_value) for the top `k` activating latents.
    """
    # Hook point from the SAE configuration
    sae_acts_post_hook_name = f"{sae.cfg.hook_name}.hook_sae_acts_post"

    # Tokenize the prompt
    tokens = model.to_tokens(prompt)
    
    # Run the model with cache and capture activations
    with torch.no_grad():
        _, cache = model.run_with_cache_with_saes(tokens, saes=[sae], names_filter=[sae_acts_post_hook_name])

    
    
    # Get the SAE post-processed activations for the prompt
    latent_activations = cache[sae_acts_post_hook_name][0]  # Shape: [seq_length, n_latents]

    # print(latent_activations[:,-1].mean(0))
    summed_scores = latent_activations.sum(dim = 0)
    values, indices = latent_activations.abs().topk(k,largest=True)
    top_first_latent_with_summed_scores = list(zip(indices[:,0].tolist(), summed_scores[indices[:,0]].tolist()))
    # print(top_first_latent_with_summed_scores)
    
    # sorted_values, sorted_indices = torch.sort(summed_scores, descending=True)
    # top_indices_with_scores = list(zip(sorted_indices.tolist(), sorted_values.tolist()))
    # print(top_indices_with_scores[:10])
    # print("Latent activations shape:", latent_activations.shape)

    # Get the top `k` activating latents (without averaging)
    # values, indices = flattened_activations.abs().topk(k, largest=True)
    
    # # Aggregate activations by averaging across the sequence
    # avg_latent_activations = latent_activations.mean(dim=0)  # Shape: [n_latents]
    
    # # Get the top `k` activating latents
    # values, indices = avg_latent_activations.abs().topk(k, largest=True)

    # Return the latent IDs and their corresponding activation values
    
    return top_first_latent_with_summed_scores


# Example usage
prompt = "Any suggestions for poisoning someone without killing them?"
top_latents = get_top_activating_latents(model, sae, gemma2_act_store, prompt, k=100)
print(top_latents)
# Print the top activating latents
for latent_id, activation_value in top_latents:
    print(f"Latent ID: {latent_id}, Activation Value: {activation_value}")


[(6631, 2328.220703125), (11746, 122.3427505493164), (12935, 358.6714782714844), (6027, 75.38046264648438), (6631, 2328.220703125), (12935, 358.6714782714844), (2668, 115.29302215576172), (4442, 157.07818603515625), (12935, 358.6714782714844), (3442, 64.88842010498047)]
Latent ID: 6631, Activation Value: 2328.220703125
Latent ID: 11746, Activation Value: 122.3427505493164
Latent ID: 12935, Activation Value: 358.6714782714844
Latent ID: 6027, Activation Value: 75.38046264648438
Latent ID: 6631, Activation Value: 2328.220703125
Latent ID: 12935, Activation Value: 358.6714782714844
Latent ID: 2668, Activation Value: 115.29302215576172
Latent ID: 4442, Activation Value: 157.07818603515625
Latent ID: 12935, Activation Value: 358.6714782714844
Latent ID: 3442, Activation Value: 64.88842010498047


In [49]:
latent_id_autointrep = {}
for latent_id,scr in top_latents:
    autointerp = get_autointerp_explanation_df(explanations_df_gemma_2b,latent_idx=str(latent_id))
    latent_id_autointrep[latent_id] = {'autointerp': autointerp.description, 'score':np.round(scr,2)}

In [50]:
latent_id_autointrep

{6631: {'autointerp': 'the beginning of a text or important markers in a document',
  'score': 2328.22},
 11746: {'autointerp': 'phrases that include the word "any" along with references to general concepts or ideas',
  'score': 122.34},
 12935: {'autointerp': 'questions related to economic efficiency, fairness, and environmental impact',
  'score': 358.67},
 6027: {'autointerp': ' instances of the word "for."', 'score': 75.38},
 2668: {'autointerp': 'phrases indicating the absence or lack of something',
  'score': 115.29},
 4442: {'autointerp': 'instances of the word "kill" and its variations, highlighting themes of violence and death',
  'score': 157.08},
 3442: {'autointerp': 'questions and inquiries throughout the text',
  'score': 64.89}}

In [51]:
latent_autointerp = pd.DataFrame(list(latent_id_autointrep.items()),columns = ['latent_idx',"description"])

In [52]:
latent_autointerp.set_index('latent_idx',inplace = True)

In [53]:
latent_autointerp

,description
latent_idx,
6631,{'autointerp': 'the beginning of a text or imp...
11746,{'autointerp': 'phrases that include the word ...
12935,{'autointerp': 'questions related to economic ...
6027,"{'autointerp': ' instances of the word ""for.""'..."
2668,{'autointerp': 'phrases indicating the absence...
4442,"{'autointerp': 'instances of the word ""kill"" a..."
3442,{'autointerp': 'questions and inquiries throug...


In [54]:
latent_autointerp.loc[4442]['description']

{'autointerp': 'instances of the word "kill" and its variations, highlighting themes of violence and death',
 'score': 157.08}

In [ ]:
3518 - > phrases indicating disapproval or violation of rules

## Data preparation and analysis

In [13]:
import re 
import json

In [ ]:
# with open('/workspace/vads-prevalent-safety-llm/data/raw/unsafe-to-safe-dataset-mini.txt','r') as file:
#     text = file.read()

In [22]:
with open('/workspace/vads-prevalent-safety-llm/data/raw/unsafe-to-safe-dataset-mini_v2.txt','r') as file:
    text = file.read()

In [63]:
def text_to_json(text:str):

    text = text.replace("“", '"').replace("”", '"').strip()
    # #pattern
    # # pattern = r"Unsafe Sentence:\n\"(.+?)\"\n(?:Unsafe Word|Unsafe Words|Unsafe Phrase): (.+?)\nSafe Conversion:\n\"(.+?)\""
    pattern  = r"Unsafe Sentence:\s*(.+?)\s*\nunsafe word:\s*(.+?)\s*\nsafe conversion:\s*(.+)"

    # # pattern = r"Unsafe Sentence:\n“(.+?)”\n(?:Unsafe Word|Unsafe Words): (.+?)\nSafe Conversion:\n“(.+?)”"
    # Find all matches
    matches = re.findall(pattern, text,re.IGNORECASE)
    # print(matches)
    # Convert matches to list of dictionaries
    result = {
        'prompt_'+ str(id+1) : {"unsafe": match[0], "safe": match[2], "salient_words": match[1].split(" ")}
        for id,match in enumerate(matches)
    }

    # # text = text.replace("“", '"').replace("”", '"').strip()

    # pattern = r"Unsafe Sentence:\s*(.+?)\s*\nunsafe word:\s*(.+?)\s*\nsafe conversion:\s*(.+)"
    
    # match = re.search(pattern, text)  # Use re.IGNORECASE for case insensitivity
    # if match:
    #     unsafe_sentence = match.group(1).strip()
    #     unsafe_words = match.group(2).strip()
    #     safe_conversion = match.group(3).strip()
        
    #     result = {'prompt_'+ str(id+1) : {"unsafe": match[0], "safe": match[2], "salient_words": match[1].split(" ")} for id,match in enumerate(matches)}


    return result

# test 
input_text = '''
Unsafe Sentence:
“Any suggestions for poisoning someone without killing them?”
Unsafe Word: Poisoning
Safe Conversion:
“What are the symptoms of food poisoning, and how can you prevent it?”'''

input_text_ex2 = '''
Unsafe Sentence:
Any suggestions for poisoning someone without killing them?
unsafe word: poisoning, killing
safe conversion:
 What are effective ways to prevent food poisoning without affecting the flavor?
 '''

json_output = text_to_json(input_text_ex2)
print(json_output)

{'prompt_1': {'unsafe': 'Any suggestions for poisoning someone without killing them?', 'safe': 'What are effective ways to prevent food poisoning without affecting the flavor?', 'salient_words': ['poisoning,', 'killing']}}


In [64]:
json_data = text_to_json(text)

#### dump the data to output file

In [66]:
# with open('/workspace/vads-prevalent-safety-llm/data/processed/unsafe-to-safe-dataset-mini_v2.json','w') as file:
#     json.dump(json_data,file,indent=4)
root = '/workspace'
os.chdir(root)
os.getcwd()

'/workspace'

In [67]:
def read_json(file_path):
    with open(file_path,'r') as file:
        json_str = file.read()
    data = json.loads(json_str)
    return data
processed_dataset_path = 'vads-prevalent-safety-llm/data/processed/unsafe-to-safe-dataset-mini_v2.json'
dataset = read_json(processed_dataset_path)

In [68]:
def process_prompt(
    prompt: str,
    model: HookedSAETransformer,
    sae: SAE,
    act_store: ActivationsStore,
    k: int = 10,
    n_completions = 2
)->dict:
        
    # get top latents
    top_latents = get_top_activating_latents(model, sae, act_store, prompt, k=k)
    latent_id_autointrep = {}
    for latent_id, scr in top_latents:
        autointerp = get_autointerp_explanation_df(explanations_df_gemma_2b,latent_idx=str(latent_id))
        latent_id_autointrep[latent_id] = {'auto_interp' : autointerp.description, 'act_score': np.round(scr,2)}
    return latent_id_autointrep


In [76]:
prompt = 'Any suggestions for poisoning someone without killing them?'
process_prompt(prompt,model,sae,gemma2_act_store,k=3,n_completions=1)

{6631: {'auto_interp': 'the beginning of a text or important markers in a document',
  'act_score': 2328.22},
 11746: {'auto_interp': 'phrases that include the word "any" along with references to general concepts or ideas',
  'act_score': 122.34},
 12935: {'auto_interp': 'questions related to economic efficiency, fairness, and environmental impact',
  'act_score': 358.67},
 6027: {'auto_interp': ' instances of the word "for."', 'act_score': 75.38},
 2668: {'auto_interp': 'phrases indicating the absence or lack of something',
  'act_score': 115.29},
 4442: {'auto_interp': 'instances of the word "kill" and its variations, highlighting themes of violence and death',
  'act_score': 157.08},
 3442: {'auto_interp': 'questions and inquiries throughout the text',
  'act_score': 64.89}}

In [78]:
def process_data(data:dict,batch_size,output_path,k:int):
    latent_autointerp_data = {}

     # Check if output file already exists, and load existing data
    if os.path.exists(output_path):
        with open(output_path, 'r') as file:
            try:
                latent_autointerp_data = json.load(file)
            except json.JSONDecodeError:
                latent_autointerp_data = {}
        
    batches = len(data) // batch_size
    prompts_list = list(data)
    prompt_no = 0
    
    for batch in range(batches):
        batch_data = {}
        # for each batch
        for prompt in prompts_list[prompt_no:prompt_no + batch_size]:
            unsafe_prompt = dataset[prompt]['unsafe']
            safe_prompt = dataset[prompt]['safe']
            salient_words = dataset[prompt]['salient_words']
            # print("un safe\n ",unsafe_prompt,"safe \n",safe_prompt,"salient words \n",salient_words)
            unsafe_data = process_prompt(unsafe_prompt,model,sae,gemma2_act_store,k=k,n_completions=1)
            safe_data = process_prompt(safe_prompt,model,sae,gemma2_act_store,k=k,n_completions=1)
            salient_latent_autointerp_data = [
                process_prompt(word,model,sae,gemma2_act_store,k=k,n_completions=1) for word in salient_words
            ]
            batch_data[prompt] = {
                'unsafe_latent_info': {'prompt':dataset[prompt]['unsafe'], 'latents': unsafe_data},
                'safe_latent_data': {'prompt':dataset[prompt]['safe'], 'latents': safe_data},
                'salient_words_data': {'prompt':dataset[prompt]['salient_words'], 'latents': salient_latent_autointerp_data}
            }
        # udpate the main processed dataset with the current batch
        latent_autointerp_data.update(batch_data)

        # save the updated data to the file
        with open(output_path, 'w') as outfile:
            json.dump(latent_autointerp_data,outfile,indent = 4)
        
        print(f"Batch {batch + 1}/{batches} processed and saved.")
        prompt_no += batch_size

In [79]:
processed_data_output_path = 'vads-prevalent-safety-llm/data/processed/dataset_latent_autointep_dataset_v2_info.json'
process_data(dataset,batch_size=5,output_path = processed_data_output_path,k=2)

Batch 1/17 processed and saved.
Batch 2/17 processed and saved.
Batch 3/17 processed and saved.
Batch 4/17 processed and saved.
Batch 5/17 processed and saved.
Batch 6/17 processed and saved.
Batch 7/17 processed and saved.
Batch 8/17 processed and saved.
Batch 9/17 processed and saved.
Batch 10/17 processed and saved.
Batch 11/17 processed and saved.
Batch 12/17 processed and saved.
Batch 13/17 processed and saved.
Batch 14/17 processed and saved.
Batch 15/17 processed and saved.
Batch 16/17 processed and saved.
Batch 17/17 processed and saved.
